In [2]:
%load_ext autoreload
%autoreload 2

from src.cleaner.auto_cleaner import clean_text
import glob
import json
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
path_list=glob.glob("../data/categorized/**/*.jsonl",recursive=True)


In [5]:
len(path_list)

10552940

In [6]:
random.shuffle(path_list)

In [205]:
line_list=[]
path=random.choice(path_list)
with open(path,"r") as f:
    #1行ずつ読み込む
    for line in f:
        text=json.loads(line)["text"]
        cleaned=clean_text(text)
        print(cleaned)
        break

文化施設や動物園のある街、上野エリアで仕事したい人にオススメのコワーキングスペース・シェアオフィスはここ!
クラウドソーシングTOP » はじめようフリーランスPowered by CrowdWroks>コワーキングスペース>文化施設や動物園のある街、上野エリアで仕事したい人にオススメのコワーキングスペース・シェアオフィスはここ!
「おいどんはフリーランスでごわす!」
「あっ!今回は西郷さんのお話?」
「惜しい!上野公園のある上野エリアがテーマだよ!」
「全国を飛び回るオレには最適なエリアだよな!」
「トラベラーにもおすすめデース!」
「アメー横も楽しいメー」
上野エリアは在来線・新幹線・東京メトロなどの駅が集中し、都内だけでなく近隣エリアからのアクセスも良好です。
そのため、住宅街やオフィス街というよりも、上野公園や上野動物園、美術館などの施設やデパート、飲食店街などのイメージが強いエリアでもあります。
そんな上野エリアで「仕事もアフターファイブも楽しみたい」人にピッタリの施設をピックアップしてみました。
上野にあるウェブ制作会社「株式会社LIG」が運営するシェアオフィス・コワーキングスペースでイベントなども盛んにおこなわれている、各種クリエイターの想像力を刺激する施設です。高めの天井に200平米という広々とした空間で作業ができ、月額会員は大きなキッチンの利用も可能、カフェなども併設し、楽しく仕事をするための様々な工夫がされている施設です。2時間600円とリーズナブルなドロップイン契約も可能なので、迷ったら試しに利用して雰囲気を知ってみるとよいでしょう。
施設名いいオフィス上野
登記登録の可否 ○(月極め会員・契約コースによる)
24時間利用 △(契約コースによる)
MC-オフィス
シェアオフィスとしてはかなりお得に始められる「エムシーオフィス上野」は、2016年1月現在入会金無料キャンペーンも実施しており、住所登記を兼ねても2万円でおつりがくる、スタートアップ企業にはとてもありがたい施設です。基本的な利用方法はフリーアドレスのオープンシート契約なので、毎回自分の好きなところに座り、気分転換をすることも可能です。ミーティングスペースは個室ではありませんが、追加利用料金もなく利用できますので、とにかく安く、とにかくキレイな施設で仕事をしてみたい、という人にはお勧めで

In [189]:
print(clean_text(text))

|モテるデートに夜景はいらない!モテるデートに夜景はいらない!|モテるデートに夜景はいらない!
◆◇◆◇女性を虜にする為の『デートテクニック』ブログ◆◇◆◇
モテるデートに夜景はいらない!
出版物(電子書籍)
メールモテ男講座
面白く学ぶシリーズ
熱い恋愛相談を受ける
モテるデートに夜景はいらない!
出版物(電子書籍)
メールモテ男講座
面白く学ぶシリーズ
熱い恋愛相談を受ける
出版物(電子書籍)
メールモテ男講座
面白く学ぶシリーズ
熱い恋愛相談を受ける
|モテるデートに夜景はいらない!
2018年9月7日
WRITER
レイ@紳士の仮面を被った熱き変態
この記事を書いている人-WRITER-
レイ@紳士の仮面を被った熱き変態
『デート関係』についての恋愛テクニックを専門とするアドバイザー。恋愛全般のアドバイスも可能ではあるが、女性との二人きりの時間をこよなく愛する変態の化身のような漢。普段は鼻くそをほじるような適当感満載な一面もありつつ、スイッチが入ると“松岡修造"のように無駄に熱くなる。
タイトル彼女を作りたければ『告白』は絶対にするなっ!
モテるデートは『男性ファースト』
表紙
【デート準備編】
概要・世間の告白理論が全く使い物にならない理由
・やってはいけない女性の口説き方
・告白しないことのメリット
・告白無しのノーリスクで女性をモノにする具体的な戦略
・女性のパーソナルスペースにスルりと侵入するスキンシップ術・失敗しない食事デートプランの組み立て順序
・女性の恋愛スイッチを入れる為の個室の選び方と流れの作り方
・女性ウケするお店の外観や雰囲気の具体例(写真で解説)
・時間帯別のお店選びのコツと使い分けの要領
・絶対に欠かすことの出来ないお店の設備のこだわりポイント
形式・縦型の実用書形式
・全5章で構成
・書籍全体の文字数は約44000字程度
・著者の事例や具体的な会話例を用いて解説・キャラクターを使用した会話形式
・3ステップで構成
・書籍全体の文字数は約52000字程度
・一項目完結型で、3～4分程度で読める文量(一部例外もあり)
・画像やイラストを多数挿入(端末によって表示の差異がありますのでご了承ください。)
対象者・友達止まり&良い人止まりの恋愛対象になれない人
・優しすぎる&真面目過ぎる人
・すぐに告白してしまう人
・クロージング(付き合う

In [171]:
print(text)

☆ベイシアマートまで徒歩約4分(約300m)、セブンイレブンまで徒歩約8分(約630m)☆


In [34]:
a="川崎市高津区でブラックでも即日融"
b="川崎市高津区でブラックでは即日融"

set(a)-set(b)

{'も'}

In [53]:
line

'{"db": "cc2021_batch63-9_0.jsonl.gz", "text": "医療脱毛をやめたい!解約の手続き方法と途中解約の注意点\\nexciteエピ\\nexciteエピ>医療脱毛の効果や回数>医療脱毛をやめたい!解約の手続き方法と途中解約の注意点\\n医療脱毛をやめたい!解約の手続き方法と途中解約の注意点\\nコースの途中で脱毛効果に満足できた、通えない事情ができてしまった、契約直後にやっぱりやめたくなったなど、医療脱毛を解約したい理由は様々です。\\nこの記事では、医療脱毛の解約方法としてクーリングオフと途中解約の違いを解説し、それぞれの条件・解約から返金までの手順のほか、脱毛コースを途中解約した場合ローンはどうなるかや大手医療脱毛クリニックごとの解約手数料・有効期限について紹介しています。\\n医療脱毛解約の種類\\n医療脱毛を解約するには、申告するタイミングや脱毛コースの消化状況によって可能な方法が2つあります。\\n施術を受ける前+契約日から8日目までに申告→クーリングオフ\\n契約後9日以上経過、または1回以上施術を受けた→途中解約\\n医療脱毛は、2017年12月に改正された特定商取引法によってクーリングオフができるようになりました。クーリングオフは期限内に申告することで契約金額が全額返金されるため、契約直後に解約したくなった場合は速やかに手続きをすすめましょう。\\n万が一クーリングオフの期間を過ぎてしまった場合でも、途中解約として解約自体はできます。\\n医療脱毛をクーリングオフする条件・解約から返金までの手順\\n医療脱毛クーリングオフの条件\\nクーリングオフは一度契約を結んだ申し込みであっても、一定期間内に申告することで解約理由にかかわらず契約を解除したり契約代金が返金される制度です。\\nクーリングオフの条件\\n契約日を1日目として8日以内に「書面」で申告する\\n契約期間が1ヶ月をこえている\\n契約金額が5万円以上\\nクーリングオフを申し込む際に特に気をつけたいのは、契約した日を1日目として8日目までに書面で申告しなければならない点です。電話やメールでも申し出ること自体は可能ですが、書面という正式な記録を残しておくことでトラブルを防ぐことができます。\\nクーリングオフの締め切りである「8日目」とは相手に届く